In [12]:
import pandas as pd
import pdb

In [ ]:
# ENCSR291TRJ - h9_de
# ENCSR559HWG - h1_de

In [77]:
def convert_encode_desc(df, col):
    """
    Convert encode desc. of a sample into something more parseable
    """
    df[col] = df[col].str.lower()
    df[col] = df[col].str.replace(', ', '_')
    df[col] = df[col].str.replace(' ', '_')
    df[col] = df[col].str.replace('-', '_')
    
    return df

In [86]:
# short read -- LIMIT TO THINGS from v29
# https://www.encodeproject.org/matrix/?type=Experiment&control_type!=*&perturbed=false&assay_title=polyA+plus+RNA-seq&replicates.library.biosample.donor.organism.scientific_name=Homo+sapiens&award.project=ENCODE&assembly=GRCh38&files.file_type=tsv&biosample_ontology.term_name!=middle+frontal+area+46&lab.title=Thomas+Gingeras%2C+CSHL&lab.title=Barbara+Wold%2C+Caltech&assay_title=total+RNA-seq&replicates.library.biosample.subcellular_fraction_term_name!=cytosol&replicates.library.biosample.subcellular_fraction_term_name!=nucleus&replicates.library.biosample.subcellular_fraction_term_name!=nucleolus&replicates.library.biosample.subcellular_fraction_term_name!=chromatin&replicates.library.biosample.subcellular_fraction_term_name!=nucleoplasm

# from narges: 
# Chose two illumina assays for now:
# Total RNA-seq
# polyA plus RNA-seq
# Only in human from ENCODE at GRCh38
# Only the ones built by Caltech
# Without any cellular component

In [96]:
df = pd.read_csv("metadata.tsv", sep='\t')
df.index = df['File accession']
df = df.loc[df['Genome annotation'] == 'V29']
df = df[['File accession', 'Experiment accession', 'Biosample term name', 'Biosample type', 'Technical replicate(s)', 'Biological replicate(s)']]
df['classification'] = 'cell_line'
df.loc[df['Biosample type']=='tissue', 'classification'] = 'tissue'
df = convert_encode_desc(df, 'Biosample term name')


# rename biosamples for DE samples
# ENCSR291TRJ - h9_de
# ENCSR559HWG - h1_de
d = {'ENCSR291TRJ': 'h9_de', 'ENCSR559HWG':'h1_de'}
for key, item in d.items():
    df.loc[df['Experiment accession'] == key, 'Biosample term name'] = item

# convert hyphenated cell line names
term_map = pd.read_csv('../lr_bulk/biosamp_term_name_map.tsv', sep='\t',
                       header=None, names=['eid', 'old_name', 'idk1', 'idk2', 'new_name'])
term_map = convert_encode_desc(term_map, 'old_name')
term_map = convert_encode_desc(term_map, 'new_name')
term_map = term_map[['old_name', 'new_name']]
term_map.drop_duplicates(inplace=True)
print(len(df.index))
df = df.merge(term_map, how='left', left_on='Biosample term name', right_on='old_name')
print(len(df.index))
df.rename({'Biosample term name': 'sample'}, axis=1, inplace=True)
df.loc[~df.new_name.isnull(), 'sample'] = df.loc[~df.new_name.isnull(), 'new_name']

# biorep
temp = df[['Experiment accession', 'sample', 'File accession']].groupby(['Experiment accession', 'sample']).count().reset_index()
temp['biorep'] = temp.groupby('sample').cumcount()+1
temp = temp[['Experiment accession', 'biorep']]
temp.biorep = temp.biorep.astype(str)
df = df.merge(temp, on='Experiment accession')

df['techrep'] = df.groupby('Experiment accession').cumcount()+1
df['hr'] = df['sample'] + '_'+ df['biorep'].astype(str) +'_'+ df.techrep.astype(str)
df = df[['File accession', 'classification', 'hr']]
df['sample'] = df.hr.str.rsplit('_', n=2, expand=True)[0]
df.to_csv("metadata_polyA_corrected.tsv", sep='\t', index=False)

510
510


In [95]:
df.loc[df.hr.duplicated(keep=False)].sort_values(by='hr')[['Experiment accession', 'Biological replicate(s)', 'techrep', 'hr']]

,Experiment accession,Biological replicate(s),techrep,hr


In [85]:
df['sample'].unique().tolist()

['activated_cd4_positive_alpha_beta_t_cell',
 'cd4_positive_alpha_beta_t_cell',
 'activated_cd8_positive_alpha_beta_memory_t_cell',
 'cd8_positive_alpha_beta_memory_t_cell',
 'naive_thymus_derived_cd4_positive_alpha_beta_t_cell',
 'activated_cd4_positive_alpha_beta_memory_t_cell',
 'naive_thymus_derived_cd8_positive_alpha_beta_t_cell',
 'cd4_positive_alpha_beta_memory_t_cell',
 'stimulated_activated_cd4_positive_alpha_beta_t_cell',
 'stimulated_activated_cd8_positive_alpha_beta_t_cell',
 'activated_naive_cd8_positive_alpha_beta_t_cell',
 'liver',
 'lung',
 'heart',
 'metanephros',
 'smooth_muscle_cell_of_the_pulmonary_artery',
 'bronchial_epithelial_cell',
 'k562',
 'pulmonary_artery_endothelial_cell',
 'urinary_bladder',
 'lung_microvascular_endothelial_cell',
 'gm12878',
 'uterus',
 'cd8_positive_alpha_beta_t_cell',
 'activated_naive_cd4_positive_alpha_beta_t_cell',
 'wtc11',
 'panc1',
 'h9_de',
 'hct116',
 'psoas_muscle',
 'h1',
 'camera_type_eye',
 'lower_lobe_of_left_lung',
 'umbi

In [83]:
df.loc[df['File accession'].duplicated(keep=False)].sort_values(by='File accession')

,File accession,classification,hr,sample


In [74]:
# long read 
import swan_vis as swan
sg = swan.read('../lr_bulk/cerberus/swan/swan.p')

Read in graph from ../lr_bulk/cerberus/swan/swan.p


In [27]:
sg.adata.obs['sample'].unique().tolist()

['caco2',
 'brain',
 'mcf10a',
 'a673',
 'lung',
 'hl60_m0',
 'colon',
 'hmec',
 'k562',
 'imr90',
 'pgp1_astro',
 'h9_de',
 'liver',
 'pc9',
 'h9_panc_progen',
 'hl60_m2_24hr',
 'vessels',
 'h1',
 'h9_neural_crest',
 'ovary',
 'gm12878',
 'muscle',
 'hl60_m1_72hr',
 'wtc11',
 'huvec',
 'hl60_m1_12hr',
 'h1_de',
 'hl60_m2_72hr',
 'calu3',
 'mcf7',
 'heart',
 'pgp1',
 'hl60_m2_12hr',
 'pc3',
 'pgp1_endo',
 'ocily7',
 'adipose',
 'hl60_m1_24hr',
 'hffc6',
 'h9',
 'panc1',
 'h9_chondro',
 'pgp1_excite_neuron',
 'adrenal gland',
 'hl60',
 'h9_osteocyte',
 'h9_panc_beta',
 'hepg2',
 'hct116',
 'kidney']

In [75]:
sg.adata.obs.loc[sg.adata.obs['sample'] =='h1_de']

,dataset,total_counts,sample,classification,health_status
index,,,,,
h1_de_1_2,h1_de_1_2,1019940.0,h1_de,cell_line,NaN
h1_de_1_3,h1_de_1_3,976011.0,h1_de,cell_line,NaN
h1_de_1_1,h1_de_1_1,820555.0,h1_de,cell_line,NaN


In [76]:
sg.adata.obs.loc[sg.adata.obs['sample'] =='h9_de']

,dataset,total_counts,sample,classification,health_status
index,,,,,
h9_de_1_2,h9_de_1_2,1894107.0,h9_de,cell_line,NaN
h9_de_1_1,h9_de_1_1,2066530.0,h9_de,cell_line,NaN
